## First Demo - Adhesion Demo

```dim_x = dim_y = 100
specs = [
    Metadata(num_processors=4),
    PottsCore(dim_x=dim_x,
              dim_y=dim_y,
              neighbor_order=2,
              boundary_x="Periodic",
              boundary_y="Periodic")
]

cell_types = ["T1", "T2", "T3"]

for ct in cell_types:
    volume_specs.param_new(ct, target_volume=25, lambda_volume=2)

contact_specs = ContactPlugin(neighbor_order=2)
for idx1 in range(len(cell_types)):
    contact_specs.param_new(type_1="Medium", type_2=cell_types[idx1], energy=16)
    for idx2 in range(idx1, len(cell_types)):
        contact_specs.param_new(type_1=cell_types[idx1], type_2=cell_types[idx2], energy=16)

adhesion_specs = AdhesionFlexPlugin(neighbor_order=2)
adhesion_specs.density_new(molecule="M1", cell_type="T1", density=1.0)
adhesion_specs.density_new(molecule="M2", cell_type="T2", density=1.0)
formula = adhesion_specs.formula_new(formula_name='Binary')
formula.param_set("M1", "M1", -10.0)
formula.param_set("M1", "M2", 0.0)
formula.param_set("M2", "M2", 10.0)

from cc3d.core.PyCoreSpecs import UniformInitializer

unif_init_specs = UniformInitializer()
unif_init_specs.region_new(width=5, pt_min=(0, 0, 0), pt_max=(dim_x, dim_y, 1),
                           cell_types=["T1", "T1", "T2", "T2", "T3"])
```

In [15]:
from Spec_UI_1 import SpecificationSetupUI

ui = SpecificationSetupUI()
display(ui.create_ui())

# Debuging: 
# Random seed should be a number generator number (positive integer)
# Reset celltype and reset pottscore needs to be fixed?
# Neighbor order can't be negative for plugins 
# Weird UI 

# Newly added features:
# Dimension were bounded now. 
# Cell Type id added. 
# Advanced potts core setting added. 

# Stretch goal: 
# Add stappables 
# energy function calculation? 

# UI
'''
1. Container white space (border too wide) px -- centered container? 
2. Spacing - for cell type 
3. left Alignment the Reset Plugin button to match with the style of rest reset buttons
4. All style goes away might related to display(ui.create_ui())

'''

# Debugging:
'''
1. Reset for Cell type (ID) and the Potts Core (num_processor) are not working.
2. Plugin Neighbor Order does not catch negative value.  
'''

# Stetch goal:
'''
1. Plugins getting the right dependencies/parameter values. 
2. Question for the plugins + initializer 
'''

'\n1. Plugins getting the right dependencies/parameter values. \n2. Question for the plugins + initializer \n'

TraitError: The 'value' trait of an IntText instance expected an int, not the NoneType None.

In [5]:
from Spec_UI import SpecificationSetupUI

ui = SpecificationSetupUI()

In [3]:
# Testing vertical tabs 
import ipywidgets as widgets
from IPython.display import display

# Create content for each tab
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()

with out1:
    print("Content of Tab 1")

with out2:
    print("Content of Tab 2")

with out3:
    print("Content of Tab 3")

# Create individual buttons instead of ToggleButtons for more control
tab1_btn = widgets.Button(description='Tab 1', layout=widgets.Layout(width='100%'))
tab2_btn = widgets.Button(description='Tab 2', layout=widgets.Layout(width='100%'))
tab3_btn = widgets.Button(description='Tab 3', layout=widgets.Layout(width='100%'))

# Put them into a VBox so they stack tightly
tabs_box = widgets.VBox([tab1_btn, tab2_btn, tab3_btn], layout=widgets.Layout(width='120px'))

# Output area
content_area = widgets.Output()

# Function to switch content
def show_tab(tab):
    content_area.clear_output()
    with content_area:
        if tab == 1:
            display(out1)
        elif tab == 2:
            display(out2)
        elif tab == 3:
            display(out3)

# Attach callbacks
tab1_btn.on_click(lambda b: show_tab(1))
tab2_btn.on_click(lambda b: show_tab(2))
tab3_btn.on_click(lambda b: show_tab(3))

# Layout the tabs and content
ui = widgets.HBox([tabs_box, content_area])

# Display
display(ui)

# Initialize first tab
show_tab(1)


In [4]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

AttributeError: 'HBox' object has no attribute 'plugins_tab'

In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
print(volume_widget.__dict__)
# print(volume_widget.widgets)  # See the actual widget objects
# print(ui.plugins_tab.plugin_widgets.keys())

{'plugin_name': 'VolumePlugin', 'plugin_class': <class 'cc3d.core.PyCoreSpecs.VolumePlugin'>, 'default_instance': <cc3d.core.PyCoreSpecs.VolumePlugin object at 0x147c37df0>, 'cell_types': ['Medium', 'T1'], 'widgets': {'active': Checkbox(value=True, description='VolumePlugin', indent=False), 'config_container': VBox(children=(VBox(children=(HBox(children=(Label(value='Medium', layout=Layout(width='120px')), Label(value='Target Volume:', layout=Layout(width='100px')), FloatText(value=25.0, layout=Layout(width='120px')), Label(value='Lambda Volume:', layout=Layout(width='100px')), FloatText(value=2.0, layout=Layout(width='120px'))), layout=Layout(padding='4px 0 4px 12px'), _dom_classes=('volume-row',)), HBox(children=(Label(value='T1', layout=Layout(width='120px')), Label(value='Target Volume:', layout=Layout(width='100px')), FloatText(value=25.0, layout=Layout(width='120px')), Label(value='Lambda Volume:', layout=Layout(width='100px')), FloatText(value=2.0, layout=Layout(width='120px')))

In [ ]:
# %run ./Spec_UI.py IGNORE

In [ ]:
# print(ui.__dict__)
# print(ui.plugins_tab.plugin_widgets)
# volume_widget = ui.plugins_tab.plugin_widgets.get('VolumePlugin')
# print(volume_widget)
# print(volume_widget.widgets)  # See the actual widget objects
print(ui.plugins_tab.plugin_widgets.keys())

dict_keys(['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'ChemotaxisPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'VolumePlugin', 'SurfacePlugin'])


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from cc3d.core.PyCoreSpecs import PLUGINS

for plugin_cls in PLUGINS:
    plugin = plugin_cls()
    print(f"Plugin: {plugin_cls.__name__}")
    for param_name in plugin.spec_dict.keys():
        # For demonstration, create a text box for each parameter
        value = plugin.spec_dict[param_name]
        input_widget = widgets.Text(
            value=str(value),
            description=param_name
        )
        display(input_widget)

Plugin: AdhesionFlexPlugin


Text(value='1', description='neighbor_order')

Text(value='[]', description='molecules')

Text(value='{}', description='densities')

Text(value='{}', description='binding_formulas')

...

Text(value='{}', description='field_specs')

Plugin: SurfacePlugin


Text(value='{}', description='params')

Plugin: VolumePlugin


Text(value='{}', description='params')

In [ ]:
from cc3d.core.PyCoreSpecs import PLUGINS
print([p.__name__ for p in PLUGINS])

['AdhesionFlexPlugin', 'BoundaryPixelTrackerPlugin', 'CellTypePlugin', 'CenterOfMassPlugin', 'ChemotaxisPlugin', 'ConnectivityGlobalPlugin', 'ConnectivityPlugin', 'ContactPlugin', 'CurvaturePlugin', 'ExternalPotentialPlugin', 'FocalPointPlasticityPlugin', 'LengthConstraintPlugin', 'MomentOfInertiaPlugin', 'NeighborTrackerPlugin', 'PixelTrackerPlugin', 'SecretionPlugin', 'SurfacePlugin', 'VolumePlugin']


In [ ]:
# Option 2
from Spec_UI import SpecificationSetupUI
ui = SpecificationSetupUI()
# Don't currently have a spepare Metadata class.
# User configuration still in JSON? (yes) or a list like "spec"
# Lots of constraints - initliazer email okie extracting dependacy relationships from the API
'''specs = [
    Metadata(),
    PottsCore(dim_x=100, dim_y=100, neighbor_order=2)
]'''

'specs = [\n    Metadata(),\n    PottsCore(dim_x=100, dim_y=100, neighbor_order=2)\n]'

In [ ]:
from Spec_UI import PottsWidget, DEFAULTS
from IPython.display import display

potts_widget = PottsWidget(DEFAULTS["PottsCore"])
display(potts_widget.create_ui())

In [ ]:
from Spec_UI import CellTypeWidget, DEFAULTS
from IPython.display import display

celltype_widget = CellTypeWidget(DEFAULTS["CellType"])
display(celltype_widget.create_ui())

In [ ]:
ui.save_to_json()  # Manual save
ui.reset_all()     # Reset everything (pass None as required argument)
config = ui.current_config()  # Get current state

Saving config: {'Metadata': {'num_processors': 2, 'debug_output_frequency': 0}, 'PottsCore': {'dim_x': 1, 'dim_y': 1, 'dim_z': 1, 'steps': 0, 'anneal': 0, 'fluctuation_amplitude': 10.0, 'fluctuation_amplitude_function': 'Min', 'boundary_x': 'NoFlux', 'boundary_y': 'NoFlux', 'boundary_z': 'NoFlux', 'neighbor_order': 1, 'debug_output_frequency': 0, 'random_seed': None, 'lattice_type': 'Cartesian', 'offset': 0, 'energy_function_calculator': None}, 'CellType': [{'Cell type': 'Medium', 'freeze': False}, {'Cell type': 'T1', 'freeze': False}], 'Plugins': {'AdhesionFlexPlugin': {'neighbor_order': 1, 'neighbor_order_error': '', 'max_distance': 3, 'max_distance_error': ''}, 'VolumePlugin': {'params': [{'CellType': 'Medium', 'target_volume': 25.0, 'lambda_volume': 2.0}, {'CellType': 'T1', 'target_volume': 25.0, 'lambda_volume': 2.0}]}}}
Saving config: {'Metadata': {'num_processors': 1, 'debug_output_frequency': 0}, 'PottsCore': {'dim_x': 1, 'dim_y': 1, 'dim_z': 1, 'steps': 0, 'anneal': 0, 'fluctu

Saving config: {'Metadata': {'num_processors': 1, 'debug_output_frequency': 0}, 'PottsCore': {'dim_x': 1, 'dim_y': 1, 'dim_z': 1, 'steps': 0, 'anneal': 0, 'fluctuation_amplitude': 10.0, 'fluctuation_amplitude_function': 'Min', 'boundary_x': 'NoFlux', 'boundary_y': 'NoFlux', 'boundary_z': 'NoFlux', 'neighbor_order': 1, 'debug_output_frequency': 0, 'random_seed': None, 'lattice_type': 'Cartesian', 'offset': 0, 'energy_function_calculator': None}, 'CellType': [{'Cell type': 'Medium', 'freeze': False}], 'Plugins': {}}
